# Global BGC metrics

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import xarray as xr

from dask.distributed import Client

import catalog
import util

xr.set_options(keep_attrs=True)

import matplotlib.pyplot as plt
import cartopy
import cartopy.crs as ccrs
import pop_tools
import numpy as np
import pandas as pd

/glade/work/kristenk/miniconda/envs/cesm-exp/lib/python3.7/site-packages/dask_jobqueue/core.py:20: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


In [3]:
ds_grid = pop_tools.get_grid('POP_gx1v7')
lons=ds_grid.TLONG
lats=ds_grid.TLAT
area = ds_grid.TAREA
area_m = area/1.e4

### Parameters

In [4]:
casename = 'g.e22.GOMIPECOIAF_JRA-1p4-2018.TL319_g17.4p2z.001'
component = 'pop'
stream = 'h'

cluster_scheduler_address = None

In [5]:
assert component in ['pop']
assert stream in ['h'] #, 'h.ecosys.nday1']

### Connect to cluster

In [6]:
if cluster_scheduler_address is None:
    cluster, client = util.get_ClusterClient()
    cluster.scale(12)
else:
    client = Client(cluster_scheduler_address)
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/kristenk/proxy/8787/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/kristenk/proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.12.206.15:33751,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/kristenk/proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


### Load the data

In [13]:
%%time

dsets = catalog.to_dataset_dict(
    case=casename,
    component=component,
    stream=stream,
)
dsets.keys()

g.e22.GOMIPECOIAF_JRA-1p4-2018.TL319_g17.4p2z.001 pop h /glade/scratch/kristenk/archive
CPU times: user 1min 13s, sys: 1.14 s, total: 1min 14s
Wall time: 1min 57s


dict_keys(['g.e22.GOMIPECOIAF_JRA-1p4-2018.TL319_g17.4p2z.001.pop.h'])

In [14]:
ds = dsets[f'{casename}.{component}.{stream}']

### Variable list

In [17]:
variables =['photoC_TOT_zint',
            'photoC_sp_zint','photoC_diat_zint',
            'photoC_diaz_zint','photoC_cocco_zint',
            'SiO2_PROD','CaCO3_PROD_zint','diaz_Nfix',
            'mesozooC','microzooC','x_graze_microzoo_zint','x_graze_mesozoo_zint',
            'POC_FLUX_100m']

In [18]:
keep_vars=['z_t','z_t_150m','dz','time_bound','TAREA','TLAT','TLONG'] + variables
ds = ds.drop([v for v in ds.variables if v not in keep_vars])

In [19]:
dt_wgt = ds.time_bound.diff('d2').squeeze()
dt_wgt /= dt_wgt.sum()
#dt_wgt

In [20]:
ds = ds.weighted(dt_wgt).mean('time')

### Make Global integrals

##### Carbon-related variables

In [21]:
cvars = ['photoC_TOT_zint', 'photoC_sp_zint','photoC_diat_zint',
         'photoC_diaz_zint','photoC_cocco_zint', 'POC_FLUX_100m',
         'CaCO3_PROD_zint','x_graze_microzoo_zint','x_graze_mesozoo_zint']

ds_glb = util.global_mean(ds, ds_grid, cvars, normalize=False).compute()

nmols_to_PgCyr = 1e-9 * 12. * 1e-15 * 365. * 86400.

for v in cvars:
    ds_glb[v] = ds_glb[v] * nmols_to_PgCyr        
    ds_glb[v].attrs['units'] = 'Pg C yr$^{-1}$'

##### Percent NPP by diatoms

In [22]:
ds_glb['diatNPPpercent'] = ds_glb.photoC_diat_zint / ds_glb.photoC_TOT_zint * 100.
ds_glb['diatNPPpercent'].attrs['units'] = '%'

##### Diatom silicification

In [23]:
tmp = (ds.SiO2_PROD.isel(z_t=slice(0,10)) * 10.).sum(dim='z_t') ## depth integral, units in mmol/m2/s
tmp = tmp * area_m ## mmol/s
tmp = tmp * 86400. * 365 * 0.001 ## mol/yr 
tmp = tmp.sum(dim=('nlon','nlat')) * 1.e-12 #Tmol/yr

ds_glb['SiO2_PROD'] = tmp
ds_glb['SiO2_PROD'].attrs['units'] = 'Tmol Si yr$^{-1}$'

##### Nitrogen fixation

In [24]:
tmp = (ds['diaz_Nfix'] * 10.).sum(dim='z_t_150m') # unit is mmol/m2/s
tmp = tmp * area_m ## mmol/s
tmp = tmp * 86400. * 365. * 0.001 * 14. * 1e-12 #convert to Tg N / yr

ds_glb['diaz_Nfix'] = tmp.sum(dim=('nlon','nlat'))
ds_glb['diaz_Nfix'].attrs['units'] = 'Tg N yr$^{-1}$'

##### Zooplankton biomass

In [25]:
tmp = (ds.microzooC * 10.).sum(dim='z_t_150m') #mmol/m2
tmp = (tmp * area_m * 0.001).sum(dim=('nlon','nlat')) ## mol
tmp = tmp.values * 1.e-15 * 12.011 #Pg C
ds_glb['microzooC'] = tmp
ds_glb['microzooC'].attrs['units'] = 'Pg C'

tmp = (ds.mesozooC * 10.).sum(dim='z_t_150m') #mmol/m2
tmp = (tmp * area_m * 0.001).sum(dim=('nlon','nlat')) ## mol
tmp = tmp.values * 1.e-15 * 12.011 #Pg C
ds_glb['mesozooC'] = tmp
ds_glb['mesozooC'].attrs['units'] = 'Pg C'

##### Zooplankton productivity as % NPP

In [34]:
ds_glb['zooprodNPPpercent'] = (ds_glb['x_graze_mesozoo_zint'] + ds_glb['x_graze_microzoo_zint'])/ds_glb['photoC_TOT_zint'] * 100.
ds_glb['zooprodNPPpercent'].attrs['units'] = '%'

### Make a table of global metrics

In [35]:
df = pd.DataFrame(columns=['Metric','unit','CESM','Obs','Reference'])

In [36]:
df.loc[0] = ['NPP',ds_glb.photoC_TOT_zint.attrs['units'],np.round(ds_glb.photoC_TOT_zint.values, 2),'45 to 55','Behrenfeld & Falkowski 1997; Carr et al., 2006']
df.loc[1] = ['POC export 100m',ds_glb.POC_FLUX_100m.attrs['units'],np.round(ds_glb.POC_FLUX_100m.values, 2),'4 to 12','DeVries & Weber, 2017']
df.loc[2] = ['%NPP by diatoms',ds_glb.diatNPPpercent.attrs['units'],np.round(ds_glb.diatNPPpercent.values, 2),'40%','Nelson et al., 1995']
df.loc[3] = ['Silicification',ds_glb.SiO2_PROD.attrs['units'],np.round(ds_glb.SiO2_PROD.values, 2),'100 to 190','Nelson et al., 1995; Holzer et al., 2014']
df.loc[4] = ['Nitrogen fixation',ds_glb.diaz_Nfix.attrs['units'],np.round(ds_glb.diaz_Nfix.values, 2),'125.6 and 222.9','Wang et al., 2019']
df.loc[5] = ['Microzooplankton biomass',ds_glb.microzooC.attrs['units'],np.round(ds_glb.microzooC.values, 2),'0.24','Buitenhuis et al., 2010']
df.loc[6] = ['Mesozooplankton biomass',ds_glb.mesozooC.attrs['units'],np.round(ds_glb.mesozooC.values, 2),'0.16 to 0.19','Buitenhuis et al., 2006; Moriarty & OBrien, 2013']
df.loc[7] = ['Zoo prod % of NPP',ds_glb.zooprodNPPpercent.attrs['units'],np.round(ds_glb.zooprodNPPpercent.values, 2),'~25%','ref?']

In [37]:
df

,Metric,unit,CESM,Obs,Reference
0,NPP,Pg C yr$^{-1}$,52.19,45 to 55,"Behrenfeld & Falkowski 1997; Carr et al., 2006"
1,POC export 100m,Pg C yr$^{-1}$,6.60,4 to 12,"DeVries & Weber, 2017"
2,%NPP by diatoms,%,39.44,40%,"Nelson et al., 1995"
3,Silicification,Tmol Si yr$^{-1}$,104.22,100 to 190,"Nelson et al., 1995; Holzer et al., 2014"
4,Nitrogen fixation,Tg N yr$^{-1}$,172.95,125.6 and 222.9,"Wang et al., 2019"
5,Microzooplankton biomass,Pg C,0.23,0.24,"Buitenhuis et al., 2010"
6,Mesozooplankton biomass,Pg C,0.40,0.16 to 0.19,"Buitenhuis et al., 2006; Moriarty & OBrien, 2013"
7,Zoo prod % of NPP,%,25.27,~25%,ref?
